# Lista 3 de Introdução à Programação e Ciência de Dados

Professor: Rafael de Pinho

Monitor: Sillas Rocha

Aluno: Iago Dantas Figueirêdo

## Instruções Gerais
- Cada questão deve ser implementada em um módulo separado, com funções reutilizáveis.
- Documente todas as funções com docstrings no estilo PEP 257 e use type hints.
- Utilize apenas as bibliotecas padrão do Python e o NumPy. Bibliotecas como ```pandas```, ```scipy``` e similares não são permitidas.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from simulations import (
    simular_precos,
    calc_retornos_simples,
    calc_retornos_log,
    sma,
    rolling_std,
)
from operations import (
    rotate_90,
    sum_subdiagonals,
    block_mat_mul,
)
from filters import (
    replace_negatives,
    local_peaks,
)

## Parte 1: Simulação de Preços e Análise de Retornos

Esta primeira parte desenvolve a simulação de dados, esse tipo de simulação possui diversos usos. Implemente todos estes exercícios no módulo "simulations.py".

### 1. Simulação de Série de Preços com Ruído Gaussiano
**Função**: simular_precos(S0: float, sigma: float, days: int) -> np.ndarray

**Descrição**: Gere uma série temporal de preços de ações de forma simplificada:

$$
S_{t+1}=S_t+\varepsilon_t, \quad \varepsilon_t \sim \mathcal{N}\left(0, \sigma^2\right) .
$$


A função deve retornar um ```np.ndarray``` de tamanho ```days + 1``` , onde o primeiro elemento é S0 e, a cada passo, soma-se um ruído normal de desvio padrão ```sigma```.

**Parâmetros**:
- ```S0```: preço inicial positivo.
- ```sigma```: desvio padrão do ruído (volatilidade).
- ```days```: número de dias a simular.

**Retorno**: ```np.ndarray``` com preços simulados.

In [3]:
# Exemplo de uso da função simular_precos

S0 = 100
sigma = 1
days = 21

precos = simular_precos(S0, sigma, days)

print(f"Preços simulados para {days} dias:")
for i, preco in enumerate(precos):
    print(f"Dia {i}: {preco:.2f}")

Preços simulados para 21 dias:
Dia 0: 100.00
Dia 1: 100.50
Dia 2: 100.36
Dia 3: 101.01
Dia 4: 102.53
Dia 5: 102.30
Dia 6: 102.06
Dia 7: 103.64
Dia 8: 104.41
Dia 9: 103.94
Dia 10: 104.48
Dia 11: 104.02
Dia 12: 103.55
Dia 13: 103.79
Dia 14: 101.88
Dia 15: 100.16
Dia 16: 99.59
Dia 17: 98.58
Dia 18: 98.89
Dia 19: 97.99
Dia 20: 96.57
Dia 21: 98.04


### 2. Cálculo de Retornos Simples e Logarítmicos

**Função**: ```calc_retornos_simples(prices: np.ndarray) -> np.ndarray```

**Descrição**: Dado um vetor de preços $P=\left[P_0, P_1, \ldots, P_n\right]$, calcule os retornos simples diários:

$$
r_t=\frac{P_t-P_{t-1}}{P_{t-1}}, \quad t=1, \ldots, n
$$

Retorne um vetor de dimensão $n$.

---

**Função**: ```calc_retornos_log(prices: np.ndarray) -> np.ndarray```

**Descrição**: Para o mesmo vetor de preços $P$, calcule os log-retornos:

$$
r_t^{\log }=\ln \left(P_t / P_{t-1}\right), \quad t=1, \ldots, n .
$$


Retorne um vetor de dimensão $n$.

In [4]:
# Exemplo de uso das funções calc_retornos_simples e calc_retornos_log

prices = np.array([100, 102, 101, 105, 107])

retornos_simples = calc_retornos_simples(prices)
retornos_log = calc_retornos_log(prices)

print("\nRetornos Simples:")
for i, retorno in enumerate(retornos_simples):
    print(f"Dia {i+1}: {retorno:.4f}")
print("\nRetornos Logarítmicos:")
for i, retorno in enumerate(retornos_log):
    print(f"Dia {i+1}: {retorno:.4f}")


Retornos Simples:
Dia 1: 0.0200
Dia 2: -0.0098
Dia 3: 0.0396
Dia 4: 0.0190

Retornos Logarítmicos:
Dia 1: 0.0198
Dia 2: -0.0099
Dia 3: 0.0388
Dia 4: 0.0189


### 3. Indicadores Móveis: Média e Volatilidade

**Função**: ```sma(returns: np.ndarray, window: int) -> np.ndarray```

**Descrição**: Implemente a Média Móvel Simples (SMA) para um vetor de retornos $r=\left[r_1, \ldots, r_n\right]$. Para cada índice $t$ a partir de $t=$ window, calcule:

$$
\mathrm{SMA}_t=\frac{1}{\text { window }} \sum_{i=t-\text{window}+1}^t r_i .
$$


Retorne um vetor de tamanho $n-\text{window}+1$.

---

**Função**: ```rolling_std(returns: np.ndarray, window: int, days_size: int = 0) -> np.ndarray```

**Descrição**: Calcule o desvio padrão móvel para o vetor de retornos $r$. Para cada $t \geq \text{window}$, defina:

$$
\bar{r}_t = \frac{1}{\text{window}} \sum_{i=t-\text{window}+1}^t r_i, \quad \sigma_t=\sqrt{\frac{1}{\text{window - days\_size }} \sum_{i=t-\text{window}+1}^t\left(r_i-\bar{r}_t\right)^2} .
$$

Retorne um vetor de tamanho $n-\text{window}+1$ . O parâmetro opcional ```days_size``` permite ajustar a normalização (por exemplo, para séries anuais, mensais, etc.).

In [5]:
# Exemplo de uso das funções sma e rolling_std

returns = np.array([0.01, 0.02, -0.01, 0.03, 0.02, -0.02, 0.01])
window = 4

sma_result = sma(returns, window)
rolling_std_result = rolling_std(returns, window)

print("\nSMA (Média Móvel Simples):")
for i, sma_value in enumerate(sma_result):
    print(f"Dia {i + window - 1}: {sma_value:.4f}")
print("\nDesvio Padrão Móvel:")
for i, std_value in enumerate(rolling_std_result):
    print(f"Dia {i + window - 1}: {std_value:.4f}")


SMA (Média Móvel Simples):
Dia 3: 0.0125
Dia 4: 0.0150
Dia 5: 0.0050
Dia 6: 0.0100

Desvio Padrão Móvel:
Dia 3: 0.0148
Dia 4: 0.0150
Dia 5: 0.0206
Dia 6: 0.0187


## Parte 2: Operações em Vetores e Matrizes

Os exercícios dessa seção exploram desde cálculos vetoriais simples até manipulações matriciais mais avançadas, sempre evitando atalhos prontos do NumPy. Implemente-os em operations.py.

### 1. Rotação de Matriz $90^{\circ}$ no Sentido Horário

**Função**: ```rotate_90(A: np.ndarray) -> np.ndarray```

**Descrição**: Dada uma matriz quadrada $A \in \mathbb{R}^{n \times n}$, implemente a rotação de $90^{\circ}$ no sentido horário sem chamar ```np.rot90```. O procedimento deve seguir dois passos:

1. Transpor $A$ (trocar linhas por colunas), obtendo $B=A^T$.
2. Inverter a ordem das colunas de $B$ (cada linha de $B$ deve ser lida de trás para frente) para formar $A_{\text{rot}}$.

**Parâmetros**:
- ```A```: matriz quadrada de formato $(n, n)$.

**Retorno**: matriz rotacionada $A_{\text{rot}} \in \mathbb{R}^{n \times n}$.

In [6]:
# Exemplo de uso da função rotate_90

matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

rotated_matrix = rotate_90(matrix)

print("\nMatriz Original:")
print(matrix)
print("\nMatriz Rotacionada 90 graus:")
print(rotated_matrix)


Matriz Original:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Matriz Rotacionada 90 graus:
[[7 4 1]
 [8 5 2]
 [9 6 3]]


### 2. Soma de Subdiagonais de uma Matriz

**Função**: ```sum_subdiagonals(A: np.ndarray, k: int) -> float```

**Descrição**: Considere $A \in \mathbb{R}^{n \times n}$ e um inteiro $k$ tal que $1 \leq k<n$. Calcule a soma dos elementos na $k$-ésima subdiagonal abaixo da diagonal principal, isto é,

$$
\sum_{i=k}^{n-1} A_{i, i-k} .
$$

Não utilize np.diag(A, -k); implemente a indexação manual (por exemplo, em um loop ou compreensão de lista).

**Parâmetros**:
- ```A```: matriz quadrada de dimensão $(n, n)$.
- ```k``` : inteiro, $1 \leq k<n$.

**Retorno**: valor escalar (float) com a soma dos elementos da subdiagonal.

In [7]:
# Exemplo de uso da função sum_subdiagonals

matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
k = 1

subdiag_sum = sum_subdiagonals(matrix, k)

print("\nMatriz Original:")
print(matrix)
print(f"\nSoma dos elementos da subdiagonal com k={k}: {subdiag_sum}")


Matriz Original:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Soma dos elementos da subdiagonal com k=1: 12.0


### 3. Multiplicação de Matrizes em Blocos

**Função**: ```block_matmul(A: np.ndarray, B: np.ndarray, block_size: int) -> np.ndarray```

**Descrição**: Implemente a multiplicação de duas matrizes $A$ e $B$, ambas de formato compatível para produto, dividindo-as em subblocos de tamanho ```block_size```. Para cada par de blocos, compute o produto parcimonioso e acumule resultados:

$$
C=A \times B, \quad C_{i, j}=\sum_k A_{i, k} \cdot B_{k, j} .
$$

A ideia é percorrer $A$ e $B$ por blocos ($i_0: i_0+$ block_size, $k_0: k_0+$ block_size), multiplicar blocos correspondentes e somar ao bloco de $C$. Não utilize ```np.dot``` ou ```A @ B``` diretamente para todo o produto, mas apenas para cada subbloco individual.

**Parâmetros**:
- ```A```, ```B```: matrizes $\in \mathbb{R}^{m \times p}$ e $\mathbb{R}^{p \times n}$, respectivamente.
- ```block_size```: inteiro $>0$ indicando o tamanho de cada subbloco quadrado.

Retorno: matriz $C \in \mathbb{R}^{m \times n}$ resultado do produto em blocos.

In [8]:
# Exemplo de uso da função block_mat_mul

A = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
B = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
block_size = 2

C = block_mat_mul(A, B, block_size)
print("\nMatriz A:")
print(A)
print("\nMatriz B:")
print(B)
print("\nProduto de Matriz em Blocos:")
print(C)


Matriz A:
[[1 2 3 4]
 [5 6 7 8]]

Matriz B:
[[1 2]
 [3 4]
 [5 6]
 [7 8]]

Produto de Matriz em Blocos:
[[ 50  60]
 [114 140]]


## Parte 3: Filtragem e Picos
A última parte reúne exercícios de tratamento de sinais e filtragem através de indexação booleana, detecção de picos. Implemente em filters.py

## 1. Substituição Condicional em Vetores

**Função**: ```replace_negatives(v: np.ndarray, new_value: float) -> np.ndarray```

**Descrição**: Dado um vetor $v=\left[v_1, \ldots, v_n\right]$, crie uma cópia e substitua cada entrada negativa por ```new_value```. Não use ```np.where```; faça indexação booleana para localizar posições onde $v_i<0$ e atribua ```new_value```.

**Parâmetros**:
- ```v``` : vetor $\in \mathbb{R}^n$.
- ```new_value```: valor escalar que substituirá cada elemento negativo.

Retorno: novo vetor onde todas as entradas negativas de $v$ foram trocadas por ```new_value```.

In [9]:
# Exemplo de uso da função replace_negatives

v = np.array([-1, 2, -3, 4, -5])
new_value = 7

v_replaced = replace_negatives(v, new_value)

print("\nVetor Original:")
print(v)
print(f"\nVetor com Negativos Substituídos por {new_value}:")
print(v_replaced)


Vetor Original:
[-1  2 -3  4 -5]

Vetor com Negativos Substituídos por 7:
[7 2 7 4 7]


### 2. Identificação de Máximos Locais em Série Temporal

**Função**: ```local_peaks(series: np.ndarray) -> Tuple[np.ndarray, np.ndarray]```

**Descrição**: Dada uma série temporal unidimensional $\left\{x_1, x_2, \ldots, x_N\right\}$ com $N \geq 3$, encontre todos os máximos locais $x_t$ tais que $x_{t-1}<x_t>x_{t+1}$, para $2 \leq t \leq N-1$. Retorne:
- ```indices```: vetor de inteiros contendo as posições $t$ em que há máximos locais.
- ```peaks```: vetor de floats com os valores $x_t$ correspondentes.

Compare manualmente cada tripla $\left(x_{t-1}, x_t, x_{t+1}\right)$.

**Parâmetros**:
- ```series```: vetor $\in \mathbb{R}^N$.

**Retorno**: tupla (```indices```, ```peaks```).

In [10]:
# Exemplo de uso da função local_peaks

series = np.array([1, 3, 2, 5, 4, 6, 5, 7, 8, 6])

indices, peaks = local_peaks(series)

print("\nSérie Original:")
print(series)
print("\nPicos Locais Encontrados:")
for index, peak in zip(indices, peaks):
    print(f"Índice: {index}, Pico: {peak}")
print("")
print("Verificando manualmente cada tripla (x_{t-1}, x_t, x_{t+1}):")
for i in indices:
    _i = i - 1
    print(f"(x_{i-1}, x_{i}, x_{i+1}) = ({series[_i-1]}, {series[_i]}, {series[_i+1]})")


Série Original:
[1 3 2 5 4 6 5 7 8 6]

Picos Locais Encontrados:
Índice: 2, Pico: 3
Índice: 4, Pico: 5
Índice: 6, Pico: 6
Índice: 9, Pico: 8

Verificando manualmente cada tripla (x_{t-1}, x_t, x_{t+1}):
(x_1, x_2, x_3) = (1, 3, 2)
(x_3, x_4, x_5) = (2, 5, 4)
(x_5, x_6, x_7) = (4, 6, 5)
(x_8, x_9, x_10) = (7, 8, 6)
